## Testing of the Graph Environment

Imports

In [1]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym
import pickle
from datetime import datetime
import random

import ray

import warnings
warnings.filterwarnings('ignore')


In [10]:
from ray.rllib.agents.dqn import DQNTrainer, DEFAULT_CONFIG, APEX_DEFAULT_CONFIG
# from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

Import the Environment

In [2]:
# sys.path.insert(0,"")

from ManhattanGraph import ManhattanGraph
from gym_graphenv.envs.GraphworldManhattan import GraphEnv

In [4]:
env_config = {'pickup_hub_index':2,
        'delivery_hub_index':3,
        'pickup_timestamp': datetime(2016, 6, 14, 22, 0, 0),
        'delivery_timestamp': datetime(2016, 6, 14, 1, 0, 0) 
}

with open('env_config.pkl', 'wb') as f:
    pickle.dump(env_config, f)

In [5]:

orders = pd.read_csv('../../data/random_orders.csv', nrows=5)
print(orders)
manhattan_graph = ManhattanGraph(filename='simple', num_hubs=70)
print(manhattan_graph.hubs)
# final_hub = env.manhattan_graph.get_nodeids_list().index(random.sample(env.hubs,1)[0])
# start_hub = env.graph.get_nodeids_list().index(random.sample(env.hubs,1)[0])
for index, row in orders.iterrows():
   
    pick_up_hub_index = ManhattanGraph.get_hub_index_by_node_index(manhattan_graph,row['pickup_node_id'])
    delivery_hub_index = ManhattanGraph.get_hub_index_by_node_index(manhattan_graph,row['delivery_node_id'])
    print(pick_up_hub_index,delivery_hub_index)
   
    # env_config = {'pickup_hub_index':pick_up_hub_index,
    #     'delivery_hub_index':delivery_hub_index,
    #     'pickup_timestamp': row['pickup_timestamp'],
    #     'delivery_timestamp': row['delivery_timestamp'] 
    # }
    # print(env_config)

   pickup_node_id  delivery_node_id     pickup_timestamp   delivery_timestamp
0            2852              2238  2016-05-19 23:04:00  2016-05-20 11:04:00
1            2324              3505  2016-01-09 05:46:00  2016-01-09 17:46:00
2             257               274  2016-03-16 14:58:00  2016-03-17 02:58:00
3            3605              2062  2016-01-17 01:49:00  2016-01-17 13:49:00
4            1131              3013  2016-03-10 20:28:00  2016-03-11 08:28:00
[42453934, 4145735059, 42440798, 1919595922, 42449029, 1692433919, 7490266268, 42440743, 42428174, 42445357, 42430253, 42445867, 42443674, 42437644, 42442937, 42446959, 42446977, 42445018, 42443534, 42435684, 42430361, 42432856, 42430375, 596775930, 42436486, 42447105, 42429342, 42450025, 42447132, 42450468, 371239958, 42444457, 42432889, 3099327970, 3099327976, 561035358, 42432985, 595352904, 42428980, 42433058, 42433066, 42427769, 42427863, 42428863, 3579432156, 1061531596, 595314119, 42442475, 595295501, 9177424868, 4244241

In [6]:
env=GraphEnv()


DB: getAvailableTrips() Execution time: 45.95755624771118 seconds
loaded 154837 trips
Initialized with 154837 taxi rides within two weeks


Run random steps without Ray Trainer

In [ ]:
def run_one_episode (env):
    env.reset()
    print("reset done")
    sum_reward = 0
    for i in range(30):
        action = env.action_space.sample()
        #print(action)
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("DELIVERY DONE! sum_reward: ",sum_reward)
            break

        print("sum_reward: ",sum_reward)
        # print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

for i in range(1): 
    run_one_episode (env)

Initialize Ray

In [7]:
ray.init()

2022-05-16 15:55:08,999	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': 'tcp://127.0.0.1:65428',
 'raylet_socket_name': 'tcp://127.0.0.1:65288',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\cosmi\\AppData\\Local\\Temp\\ray\\session_2022-05-16_15-55-05_053040_6636',
 'metrics_export_port': 65332,
 'gcs_address': '127.0.0.1:65396',
 'address': '127.0.0.1:65396',
 'node_id': '4db9e4f79fe9ac8cf11aa411eb6178465ed3d08a0633dff34bfaf12b'}

Set DQN trainer configuration

In [11]:
dqn_config = DEFAULT_CONFIG.copy()
dqn_config['num_workers'] = 1
dqn_config["train_batch_size"] = 400
dqn_config["gamma"] = 0.95
dqn_config["n_step"] = 10
dqn_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#dqn_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

Set DQN - Rainbow configuration

In [ ]:
rainbow_config = DEFAULT_CONFIG.copy()

rainbow_config = DEFAULT_CONFIG.copy()
rainbow_config['num_workers'] = 1
rainbow_config["train_batch_size"] = 400
rainbow_config["gamma"] = 0.95
rainbow_config["n_step"] = 10
rainbow_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#rainbow_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

#rainbow parameters

# N-step Q learning
rainbow_config["n_step"]= 5 #[between 1 and 10]
# Whether to use noisy network
rainbow_config["noisy"] = True
# Number of atoms for representing the distribution of return. When
# this is greater than 1, distributional Q-learning is used.
# the discrete supports are bounded by v_min and v_max
rainbow_config["num_atoms"] =2 #[more than 1]
rainbow_config["v_min"] =-100
rainbow_config["v_max"]=100 # (set v_min and v_max according to your expected range of returns).

Set DQN - Ape-X configuration 

In [ ]:
#apex_config=APEX_DEFAULT_CONFIG.copy()

Initialize the DQN trainer, Rainbow DQN and DQN Ape-X

In [ ]:
ppo_trainer=PPOTrainer(DEFAULT_CONFIG,GraphEnv)

In [12]:
trainer_dqn = DQNTrainer(dqn_config,GraphEnv)

2022-05-16 15:55:49,936	INFO simple_q.py:154 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2022-05-16 15:55:49,937	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=2388) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=2388) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=2388) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=2388) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future rel

 pid=2388) DB: getAvailableTrips() Execution time: 48.90804982185364 seconds
 pid=2388) loaded 154837 trips
 pid=2388) Initialized with 154837 taxi rides within two weeks
 pid=2388) Started Reset() without config
 pid=2388) Reset initialized pickup: 29
 pid=2388) Reset initialized dropoff: 63
 pid=2388) Reset initialized time: 2016-01-04 02:37:00


2022-05-16 15:57:40,664	INFO trainable.py:127 -- Trainable.setup took 110.731 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-05-16 15:57:40,667	WARNING util.py:55 -- Install gputil for GPU system monitoring.


 pid=2388) Get available trips in step took 0.011016845703125 s, found 0 trips
 pid=2388) Reset() Execution Time: 50.92469668388367


In [ ]:
trainer_rainbow=DQNTrainer(rainbow_config,GraphEnv )


In [ ]:
#trainer_apex=DQNTrainer(apex_config,GraphEnv )

Define the path where the results of the trainer should be saved

In [13]:
checkpoint_root = "tmp/dqn/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Run the trainer

In [14]:
results = []
episode_data = []
episode_json = []
n_iter=1

for n in range(n_iter):
    result = trainer_dqn.train()
    results.append(result)
    print("TEST",n)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = trainer_dqn.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

 pid=2388) Started Reset() without config
 pid=2388) Reset initialized pickup: 28
 pid=2388) Reset initialized dropoff: 64
 pid=2388) Reset initialized time: 2016-01-04 02:37:00
 pid=2388) Get available trips in step took 0.019968509674072266 s, found 0 trips
 pid=2388) Reset() Execution Time: 0.18204855918884277
 pid=2388) Time Ride: 0.012984991073608398
 pid=2388) Get available trips in step took 0.012951850891113281 s, found 0 trips
 pid=2388) 28 -> 57 8094.714
 pid=2388) Time Ride: 0.03096151351928711
 pid=2388) Get available trips in step took 0.016999483108520508 s, found 0 trips
 pid=2388) 57 -> 40 16432.770000000004
 pid=2388) Time Ride: 0.014983654022216797
 pid=2388) Get available trips in step took 0.015998363494873047 s, found 0 trips
 pid=2388) 40 -> 28 9846.050000000001
 pid=2388) Time Ride: 0.017986297607421875
 pid=2388) Get available trips in step took 0.01497507095336914 s, found 0 trips
 pid=2388) 28 -> 40 10058.664000000004
 pid=2388) Time Ride: 0.021975040435791016

2022-05-16 16:00:07,818	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


 pid=2388) Time Ride: 0.029013633728027344
 pid=2388) Get available trips in step took 0.011009931564331055 s, found 0 trips
 pid=2388) 56 -> 44 13586.711000000007
TEST 0
  1: Min/Mean/Max reward: -10728.1337/-4062.4042/961.8736, len mean:  47.4762. Checkpoint saved to tmp/dqn/graphworld\checkpoint_000001\checkpoint-1


In [ ]:
results

Testing of the trainer

In [16]:
restore_trainer = DQNTrainer(dqn_config,GraphEnv)
restore_trainer.restore(file_name)
# env = gym.make("graphworld-v0")
state = env.reset()

 pid=11696) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=11696) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=11696) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=11696) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=11696) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=11696) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the

 pid=11696) DB: getAvailableTrips() Execution time: 46.54477000236511 seconds
 pid=11696) loaded 154837 trips
 pid=11696) Initialized with 154837 taxi rides within two weeks
 pid=11696) Started Reset() without config
 pid=11696) Reset initialized pickup: 67
 pid=11696) Reset initialized dropoff: 60
 pid=11696) Reset initialized time: 2016-01-03 08:34:00


2022-05-16 16:21:48,778	INFO trainable.py:127 -- Trainable.setup took 104.894 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-05-16 16:21:48,783	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-05-16 16:21:48,805	INFO trainable.py:495 -- Restored on 127.0.0.1 from checkpoint: tmp/dqn/graphworld\checkpoint_000001\checkpoint-1
2022-05-16 16:21:48,806	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': 400, '_time_total': 90.0609827041626, '_episodes_total': 21}


 pid=11696) Get available trips in step took 0.011999130249023438 s, found 0 trips
 pid=11696) Reset() Execution Time: 48.425469636917114
Started Reset() without config
Reset initialized pickup: 34
Reset initialized dropoff: 12
Reset initialized time: 2016-01-08 11:40:00
Get available trips in step took 0.01200103759765625 s, found 0 trips
Reset() Execution Time: 55.655370473861694


In [17]:
sum_reward = 0
n_step = 5
for step in range(n_step):
    action = restore_trainer.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == True:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

2022-05-16 16:33:46,408	WARNING deprecation.py:45 -- DeprecationWarning: `compute_action` has been deprecated. Use `Trainer.compute_single_action()` instead. This will raise an error in the future!


Time Ride: 0.01798272132873535
Get available trips in step took 0.014992713928222656 s, found 0 trips
34 -> 49 6513.563000000001
Time Ride: 0.014997005462646484
Get available trips in step took 0.016021728515625 s, found 0 trips
49 -> 40 10061.127
Time Ride: 0.028954029083251953
Get available trips in step took 0.014055728912353516 s, found 0 trips
40 -> 0 20441.967000000008
Time Ride: 0.008000612258911133
Get available trips in step took 0.01399374008178711 s, found 0 trips
0 -> 11 4336.3769999999995
Time Ride: 0.01598811149597168
Get available trips in step took 0.013971805572509766 s, found 0 trips
11 -> 46 8498.421


In [ ]:
ray.shutdown()